# Notebook to interactively manipulate equilibrium boundary and getting corresponding MXH coefficients

In [ ]:
using FUSE
using Plots; gr();
using IMAS

In [ ]:
using Interact

my_mxh = Dict()
upnull = false
lonull = false
@manipulate for mhx_order in 1:10,
    a in LinRange(0.1, 10, 101),
    eps in LinRange(0.1, 1, 101),
    kapu in LinRange(1.0, 3.0, 101),
    kapl in LinRange(1.0, 3.0, 101),
    delu in LinRange(-1.0, 1.0, 101),
    dell in LinRange(-1.0, 1.0, 101),
    zetaou in LinRange(-sqrt(2) / 2, 1.0, 101),
    zetaiu in LinRange(-sqrt(2) / 2, 1.0, 101),
    zetaol in LinRange(-sqrt(2) / 2, 1.0, 101),
    zetail in LinRange(-sqrt(2) / 2, 1.0, 101)#,
    #upnull in [false,true],
    #lonull in [false,true],

    r, z, zref = try
        IMAS.boundary_shape(; a, eps, kapu, kapl, delu, dell, zetaou, zetaiu, zetail, zetaol, zoffset=0.0, upnull, lonull)
    catch e
        [NaN], [NaN], [NaN]
    end
    my_mxh["current"] = mxh = IMAS.MXH(r, z, mhx_order)

    plot(r, z; aspect_ratio=:equal, lw=2, label="numerical")
    plot!(mxh,label="MXH")
end

In [ ]:
mhx_order = 4

R0 = 5.0
eps = 1/3.5
a = R0*eps
kapu = 2.0
kapl = 2.2
delu = 0.5
dell = 0.7
zetaou = -0.0
zetaol = -0.2
zetaiu = -0.0
zetail = -0.2
upnull = false
lonull = false

r, z, zref = try
    IMAS.boundary_shape(; a, eps, kapu, kapl, delu, dell, zetaou, zetaiu, zetaol, zetail, zoffset=0.0, upnull, lonull)
catch e
    [NaN], [NaN], [NaN]
end

mxh = IMAS.MXH(r, z, mhx_order)
plot(r, z; aspect_ratio=:equal, lw=2, label="numerical")
display(plot!(mxh,label="MXH"))
display(mxh)

In [ ]:
mxh_vect = IMAS.flat_coeffs(mxh)

In [ ]:
# CHEASE deadstart MXH
dd = IMAS.dd()
ini, act = FUSE.case_parameters(:ITER, init_from=:scalars);
ini.equilibrium.boundary_from = :MXH_params
ini.equilibrium.MXH_params = mxh_vect
ini.equilibrium.xpoints_number = 1
act.ActorEquilibrium.model = :CHEASE

act

In [ ]:
dd = FUSE.init(dd, ini, act)
mxh_plot = plot(dd.equilibrium, label="CHEASE deadstart MXH")
display(mxh_plot)